In [1]:
import requests
import csv
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep
from urllib.parse import urljoin
import os
import snowflake.connector
from datetime import datetime
import uuid
import re
import pandas as pd
import time
from word2number import w2n  # Library to convert numbers to words

In [2]:
URL = 'https://www.espncricinfo.com/records/trophy/indian-premier-league-117'
sleep_delay = 5
#https://rforotb-ka22072.snowflakecomputing.com
# Snowflake connection parameters
snowflake_account = 'rforotb-ka22072'
snowflake_warehouse= 'COMPUTE_WH'
snowflake_database = 'MYCRICKET'
snowflake_schema = 'DATA1'
snowflake_user = 'comicsvibe'
snowflake_password = 'Comicsvibe123!'
snowflake_stage = 'MYSTAGE1'

# Establish Snowflake connection
conn = snowflake.connector.connect(
    account=snowflake_account,
    warehouse=snowflake_warehouse,
    database=snowflake_database,
    schema=snowflake_schema,
    user=snowflake_user,
    password=snowflake_password,
    stage=snowflake_stage
)

In [3]:
# Create a cursor to execute SQL statements
cursor = conn.cursor()

# List the target table names in your Snowflake schema
cursor.execute(f"SHOW TABLES IN {snowflake_schema}")
table_names = [row[1] for row in cursor.fetchall()]
print(table_names)

# List the CSV files in the Snowflake stage
cursor.execute(f"LIST @{snowflake_stage}")
FILE_NAMES = [row[0] for row in cursor.fetchall()]
print(FILE_NAMES)

# Upload the CSV data from the Snowflake stage to the matching tables in the schema
for file_name in FILE_NAMES:
    # Remove the file extension from the CSV file name to compare with table names
    file_name_without_ext = file_name.split("/")[-2]
    print(file_name_without_ext)
    file_name_without_ext1 = file_name_without_ext.split(".")[0]
    print(file_name_without_ext1)
    file_name_upper = file_name_without_ext1.upper()

    # Check if the file name matches any of the table names
    if file_name_upper in table_names:
        try:
            # Use the TRUNCATE command to delete all records from the table
            truncate_query = f"TRUNCATE TABLE {snowflake_schema}.{file_name_upper}"
            cursor.execute(truncate_query)

            # Use the COPY INTO command to load data from the stage to the table
            copy_into_query = f"""
            COPY INTO {snowflake_schema}.{file_name_upper}
            FROM @{snowflake_stage}/{file_name_without_ext}
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            """
            print(copy_into_query)
            cursor.execute(copy_into_query)
            print(f"Data uploaded from {file_name} to {snowflake_schema}.{file_name_upper}.")
        except snowflake.connector.errors.ProgrammingError as e:
            # Catch the exception when COPY INTO reaches the end of the CSV file
            # (Error code 100080: Number of columns in file does not match that of the corresponding table)
            if e.errno == 100080:
                print(f"Finished loading data from {file_name} into {snowflake_schema}.{file_name_upper}.")
            else:
                # If it's a different ProgrammingError, raise it to handle other potential issues
                raise e

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()

['ABANDONED_MATCHES_283285', 'BEST_CAREER_BOWLING_AVERAGE_283258', 'BEST_CAREER_BOWLING_AVERAGE_WITHOUT_QUALIFICATION_283899', 'BEST_CAREER_ECONOMY_RATE_283267', 'BEST_CAREER_STRIKE_RATE_283276', 'BEST_ECONOMY_RATE_IN_AN_INNINGS_283890', 'BEST_FIGURES_IN_AN_INNINGS_283973', 'BEST_FIGURES_IN_A_INNINGS_BY_A_CAPTAIN_283935', 'BEST_FIGURES_IN_A_INNINGS_ON_DEBUT_283833', 'BEST_FIGURES_IN_A_INNINGS_WHEN_ON_THE_LOSING_SIDE_283926', 'BEST_STRIKE_RATE_IN_AN_INNINGS_283886', 'BOWLER_BATTERS_COMBINATIONS_283734', 'BOWLER_FIELDER_COMBINATIONS_283728', 'CAPTAINS_WHO_HAVE_KEPT_WICKET_283402', 'CAPTAINS_WHO_HAVE_KEPT_WICKET_AND_OPENED_THE_BATTING_283396', 'CARRYING_BAT_THROUGH_A_COMPLETED_INNINGS_284000', 'DISMISSED_BY_THE_FIRST_BALL_OF_A_MATCH_283991', 'DISMISSED_FOR_99_569883', 'EXTRAS_BEING_HIGHEST_SCORER_IN_A_COMPLETED_INNINGS_282984', 'FASTEST_FIFTIES_284094', 'FASTEST_HUNDREDS_386083', 'FASTEST_TO_1000_RUNS_925247', 'FASTEST_TO_100_WICKETS_1286053', 'FASTEST_TO_2000_RUNS_1213211', 'FASTEST_TO_2


            COPY INTO DATA3.ABANDONED_MATCHES_283285
            FROM @MYSTAGE3/abandoned_matches_283285.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/abandoned_matches_283285.csv/abandoned_matches_283285.csv.gz to DATA3.ABANDONED_MATCHES_283285.
best_career_bowling_average_283258.csv
best_career_bowling_average_283258

            COPY INTO DATA3.BEST_CAREER_BOWLING_AVERAGE_283258
            FROM @MYSTAGE3/best_career_bowling_average_283258.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/best_career_bowling_average_283258.csv/best_career_bowling_average_283258.csv.gz to DATA3.BEST_CAREER_BOWLING_AVERAGE_283258.
best_career_bowling_average_without_qualific

Data uploaded from mystage3/captains_who_have_kept_wicket_and_opened_the_batting_283396.csv/captains_who_have_kept_wicket_and_opened_the_batting_283396.csv.gz to DATA3.CAPTAINS_WHO_HAVE_KEPT_WICKET_AND_OPENED_THE_BATTING_283396.
carrying_bat_through_a_completed_innings_284000.csv
carrying_bat_through_a_completed_innings_284000

            COPY INTO DATA3.CARRYING_BAT_THROUGH_A_COMPLETED_INNINGS_284000
            FROM @MYSTAGE3/carrying_bat_through_a_completed_innings_284000.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/carrying_bat_through_a_completed_innings_284000.csv/carrying_bat_through_a_completed_innings_284000.csv.gz to DATA3.CARRYING_BAT_THROUGH_A_COMPLETED_INNINGS_284000.
dismissed_by_the_first_ball_of_a_match_283991.csv
dismissed_by_the_first_ball_of_a_match_283991

            COPY INTO DATA3.DISMISSED_BY_TH

Data uploaded from mystage3/fifties_in_consecutive_innings_284104.csv/fifties_in_consecutive_innings_284104.csv.gz to DATA3.FIFTIES_IN_CONSECUTIVE_INNINGS_284104.
four_wickets_in_four_balls_1199858.csv
four_wickets_in_four_balls_1199858

            COPY INTO DATA3.FOUR_WICKETS_IN_FOUR_BALLS_1199858
            FROM @MYSTAGE3/four_wickets_in_four_balls_1199858.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/four_wickets_in_four_balls_1199858.csv/four_wickets_in_four_balls_1199858.csv.gz to DATA3.FOUR_WICKETS_IN_FOUR_BALLS_1199858.
hat_tricks_283738.csv
hat_tricks_283738

            COPY INTO DATA3.HAT_TRICKS_283738
            FROM @MYSTAGE3/hat_tricks_283738.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CO

Data uploaded from mystage3/highest_partnership_for_the_fourth_wicket_283587.csv/highest_partnership_for_the_fourth_wicket_283587.csv.gz to DATA3.HIGHEST_PARTNERSHIP_FOR_THE_FOURTH_WICKET_283587.
highest_partnership_for_the_ninth_wicket_283551.csv
highest_partnership_for_the_ninth_wicket_283551

            COPY INTO DATA3.HIGHEST_PARTNERSHIP_FOR_THE_NINTH_WICKET_283551
            FROM @MYSTAGE3/highest_partnership_for_the_ninth_wicket_283551.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/highest_partnership_for_the_ninth_wicket_283551.csv/highest_partnership_for_the_ninth_wicket_283551.csv.gz to DATA3.HIGHEST_PARTNERSHIP_FOR_THE_NINTH_WICKET_283551.
highest_partnership_for_the_second_wicket_283601.csv
highest_partnership_for_the_second_wicket_283601

            COPY INTO DATA3.HIGHEST_PARTNERSHIP_FOR_THE_SECOND_WICKET_

...

Data uploaded from mystage3/most_runs_from_fours_and_sixes_in_an_innings_283004.csv/most_runs_from_fours_and_sixes_in_an_innings_283004.csv.gz to DATA3.MOST_RUNS_FROM_FOURS_AND_SIXES_IN_AN_INNINGS_283004.
most_runs_from_fours_and_sixes_in_an_innings_284015.csv
most_runs_from_fours_and_sixes_in_an_innings_284015

            COPY INTO DATA3.MOST_RUNS_FROM_FOURS_AND_SIXES_IN_AN_INNINGS_284015
            FROM @MYSTAGE3/most_runs_from_fours_and_sixes_in_an_innings_284015.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/most_runs_from_fours_and_sixes_in_an_innings_284015.csv/most_runs_from_fours_and_sixes_in_an_innings_284015.csv.gz to DATA3.MOST_RUNS_FROM_FOURS_AND_SIXES_IN_AN_INNINGS_284015.
most_runs_in_a_calendar_year_284247.csv
most_runs_in_a_calendar_year_284247

            COPY INTO DATA3.MOST_RUNS_IN_A_CALENDAR_YEAR_28

Data uploaded from mystage3/most_sixes_in_a_match_283012.csv/most_sixes_in_a_match_283012.csv.gz to DATA3.MOST_SIXES_IN_A_MATCH_283012.
most_sixes_in_an_innings_283019.csv
most_sixes_in_an_innings_283019

            COPY INTO DATA3.MOST_SIXES_IN_AN_INNINGS_283019
            FROM @MYSTAGE3/most_sixes_in_an_innings_283019.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/most_sixes_in_an_innings_283019.csv/most_sixes_in_an_innings_283019.csv.gz to DATA3.MOST_SIXES_IN_AN_INNINGS_283019.
most_sixes_in_an_innings_284019.csv
most_sixes_in_an_innings_284019

            COPY INTO DATA3.MOST_SIXES_IN_AN_INNINGS_284019
            FROM @MYSTAGE3/most_sixes_in_an_innings_284019.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERR

Data uploaded from mystage3/oldest_players_283455.csv/oldest_players_283455.csv.gz to DATA3.OLDEST_PLAYERS_283455.
oldest_players_on_debut_283460.csv
oldest_players_on_debut_283460

            COPY INTO DATA3.OLDEST_PLAYERS_ON_DEBUT_283460
            FROM @MYSTAGE3/oldest_players_on_debut_283460.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/oldest_players_on_debut_283460.csv/oldest_players_on_debut_283460.csv.gz to DATA3.OLDEST_PLAYERS_ON_DEBUT_283460.
outstanding_bowling_analyses_in_an_innings_283952.csv
outstanding_bowling_analyses_in_an_innings_283952

            COPY INTO DATA3.OUTSTANDING_BOWLING_ANALYSES_IN_AN_INNINGS_283952
            FROM @MYSTAGE3/outstanding_bowling_analyses_in_an_innings_283952.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-

Data uploaded from mystage3/unusual_dismissals_283988.csv/unusual_dismissals_283988.csv.gz to DATA3.UNUSUAL_DISMISSALS_283988.
wicket_with_first_ball_in_career_530931.csv
wicket_with_first_ball_in_career_530931

            COPY INTO DATA3.WICKET_WITH_FIRST_BALL_IN_CAREER_530931
            FROM @MYSTAGE3/wicket_with_first_ball_in_career_530931.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE, NULL_IF = ('-'), VALIDATE_UTF8 = TRUE)
            ON_ERROR = 'CONTINUE'
            
Data uploaded from mystage3/wicket_with_first_ball_in_career_530931.csv/wicket_with_first_ball_in_career_530931.csv.gz to DATA3.WICKET_WITH_FIRST_BALL_IN_CAREER_530931.
winning_all_tosses_in_a_series_1223147.csv
winning_all_tosses_in_a_series_1223147

            COPY INTO DATA3.WINNING_ALL_TOSSES_IN_A_SERIES_1223147
            FROM @MYSTAGE3/winning_all_tosses_in_a_series_1223147.csv
            FILE_FORMAT = (TYPE = CSV, SKIP_HEADER = 1, ERROR_ON_COLUMN_COUNT_

True